In [1]:
import keras_models as kms
import keras_forward as kf
import np_models as npms
import np_forward as npf
import tf_graphs as tfgs
import tf_forward as tff
import tools
import PyPolyChord
import PyPolyChord.settings
import inverse_priors
import polychord_tools
import output
import input_tools
import forward_tests
import prior_tests

/home/astrophysics/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
###### load training data
data = 'simple_tanh'
data_dir = '../../data/'
data_prefix = data_dir + data
x_tr, y_tr = input_tools.get_x_y_tr_data(data_prefix)
batch_size = x_tr.shape[0]

In [3]:
###### get weight information
a1_size = 2
layer_sizes = [a1_size]
m_trainable_arr = [True, False]
b_trainable_arr = [False, False]
num_inputs = tools.get_num_inputs(x_tr)
num_outputs = tools.get_num_outputs(y_tr)
num_weights = tools.calc_num_weights3(num_inputs, layer_sizes, num_outputs, m_trainable_arr, b_trainable_arr)

In [4]:
###### check shapes of training data
x_tr, y_tr = tools.reshape_x_y_twod(x_tr, y_tr)

In [5]:
###### setup keras model
model = kms.slp_model(num_inputs, num_outputs, layer_sizes)
km = kf.keras_model(model, x_tr, y_tr, batch_size)
loss = 'mse' 
km.setup_LL(loss)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
#setup tf graph
tf_graph = tfgs.slp_graph
tfm = tff.tf_model(tf_graph, x_tr, y_tr, batch_size, layer_sizes, m_trainable_arr, b_trainable_arr)
fit_metric = 'chisq'
tfm.setup_LL(fit_metric)

[(1, 2)]


In [7]:
#set up np model
np_nn = npms.slp_nn
npm = npf.np_model(np_nn, x_tr, y_tr, batch_size, layer_sizes, m_trainable_arr, b_trainable_arr)
ll_type = 'gauss'
npm.setup_LL(ll_type)

In [8]:
#time forward_propagation using keras model
%timeit forward_tests.forward_test_linear([km], num_weights, data_dir, print_out = False)

The slowest run took 21.82 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 2.68 ms per loop


In [9]:
#time forward_propagation using tensorflow model
# import tensorflow as tf
# sess = tf.Session() 
%timeit forward_tests.forward_test_linear([tfm], num_weights, data_dir, print_out = False)

The slowest run took 14.52 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 994 µs per loop


In [10]:
#time forward_propagation using np model
%timeit forward_tests.forward_test_linear([npm], num_weights, data_dir, print_out = False)

The slowest run took 21.44 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 542 µs per loop


In [12]:
###### setup prior
prior_types = [7]
prior_hyperparams = [[-1., 1.]]
weight_shapes = tools.get_weight_shapes3(num_inputs, layer_sizes, num_outputs, m_trainable_arr, b_trainable_arr)
dependence_lengths = tools.get_degen_dependence_lengths(weight_shapes)
param_prior_types = [0]
prior = inverse_priors.inverse_prior(prior_types, prior_hyperparams, dependence_lengths, param_prior_types, num_weights)

In [13]:
###### time prior output from nn setup
%timeit prior_tests.nn_prior_test(prior, print_out = False)

The slowest run took 15.85 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 298 µs per loop


In [14]:
###### setup polychord
nDerived = 0
settings = PyPolyChord.settings.PolyChordSettings(num_weights, nDerived)
settings.file_root = data
settings.nlive = 200
settings.feedback = 0
settings.posteriors = False
settings.write_stats = False

In [12]:
#time polychord with keras model
settings.base_dir = './keras_chains/'
%timeit PyPolyChord.run_polychord(km, num_weights, nDerived, settings, prior, polychord_tools.dumper)

('Last dead point:', array([-4.79972461e-02,  2.63971361e-02, -9.20693729e+02, -9.20654022e+02]))
('Last dead point:', array([-2.60034000e-01,  3.09973331e-01, -1.00000000e+30, -9.19272975e+02]))
('Last dead point:', array([-1.81389210e-01,  3.64880144e-01, -9.20172262e+02, -9.19030863e+02]))
('Last dead point:', array([-2.53652080e-01,  9.86464494e-01, -9.18990395e+02, -9.18960769e+02]))
('Last dead point:', array([-2.14496908e-01,  7.13642565e-01, -9.18970040e+02, -9.18944837e+02]))
('Last dead point:', array([-1.98588001e-01,  6.97004719e-01, -9.18941972e+02, -9.18941034e+02]))
('Last dead point:', array([-2.08918946e-01,  8.35714163e-01, -9.18939565e+02, -9.18939381e+02]))
('Last dead point:', array([-1.94742550e-01,  7.81104254e-01, -9.18939338e+02, -9.18938859e+02]))
('Last dead point:', array([-2.00153226e-01,  7.99143046e-01, -9.18980155e+02, -9.18938534e+02]))
('Last dead point:', array([-5.28655467e-02,  9.48315709e-01, -2.90916564e+03, -9.20644973e+02]))
('Last dead point:',

In [13]:
#time polychord with tf model
settings.base_dir = './tf_chains/'
%timeit PyPolyChord.run_polychord(tfm, num_weights, nDerived, settings, prior, polychord_tools.dumper)

('Last dead point:', array([-9.47504741e-01,  3.18080815e-01, -1.00000000e+30, -9.20225230e+02]))
('Last dead point:', array([-1.01024258e-01,  3.49613711e-01, -9.19338808e+02, -9.19211135e+02]))
('Last dead point:', array([-1.66025051e-01,  3.75411574e-01, -9.19145865e+02, -9.19012787e+02]))
('Last dead point:', array([-2.47014538e-01,  8.82633951e-01, -9.18992140e+02, -9.18956841e+02]))
('Last dead point:', array([-2.22840946e-01,  9.13054173e-01, -9.18955668e+02, -9.18943776e+02]))
('Last dead point:', array([-2.07511325e-01,  7.46887938e-01, -9.18940529e+02, -9.18940480e+02]))
('Last dead point:', array([-1.92927227e-01,  7.94536100e-01, -9.18939476e+02, -9.18939196e+02]))
('Last dead point:', array([-2.03368945e-01,  8.34780564e-01, -9.18938824e+02, -9.18938768e+02]))
('Last dead point:', array([-1.99503285e-01,  7.97853891e-01, -9.18939074e+02, -9.18938536e+02]))
('Last dead point:', array([-7.47306000e-01,  2.62788865e-01, -1.00000000e+30, -9.20324624e+02]))
('Last dead point:',

In [14]:
#time polychord with np model
settings.base_dir = './np_chains/'
%timeit PyPolyChord.run_polychord(npm, num_weights, nDerived, settings, prior, polychord_tools.dumper)

('Last dead point:', array([-5.99223859e-02,  3.92134697e-01, -1.00000000e+30, -9.20110741e+02]))
('Last dead point:', array([-4.09652669e-01,  5.32529441e-01, -9.19533374e+02, -9.19253416e+02]))
('Last dead point:', array([-3.30481942e-01,  9.01599550e-01, -9.19106633e+02, -9.19030376e+02]))
('Last dead point:', array([-2.59148060e-01,  9.77312694e-01, -9.20417321e+02, -9.18964501e+02]))
('Last dead point:', array([-2.26939130e-01,  8.63205525e-01, -9.18947950e+02, -9.18945299e+02]))
('Last dead point:', array([-2.10640866e-01,  7.59553678e-01, -9.18943703e+02, -9.18940918e+02]))
('Last dead point:', array([-2.08495522e-01,  8.07022566e-01, -9.18939934e+02, -9.18939376e+02]))
('Last dead point:', array([-2.00072964e-01,  7.64850126e-01, -9.18939097e+02, -9.18938811e+02]))
('Last dead point:', array([-1.99953825e-01,  8.00323725e-01, -9.18938859e+02, -9.18938533e+02]))
('Last dead point:', array([-6.13844761e-01,  2.62519642e-01, -2.78512704e+03, -9.20171751e+02]))
('Last dead point:',

In [ ]:
import pstats, cProfile

In [1]:
cProfile.runctx(function to be profiled, globals(), locals(), name of prof file)

s = pstats.Stats(name of prof file)
s.strip_dirs().sort_stats("time").print_stats()


SyntaxError: invalid syntax (<ipython-input-1-2674c9a83d50>, line 1)